In [31]:
import numpy as np
import pandas as pd
import scipy.linalg as la

In [31]:
test_matrix = np.array([[1, 2], [3, 4]])
big_test_matrix = la.block_diag(test_matrix, test_matrix)
readable_big_test_matrix = pd.DataFrame(big_test_matrix)
print(type(big_test_matrix))

<class 'numpy.ndarray'>


In [33]:
def kron(A, B):
    return la.kron(A, B)

In [34]:
def kronsum(A, B):
    if A.shape[0] != A.shape[1]:
        raise ValueError('A is not square')

    if B.shape[0] != B.shape[1]:
        raise ValueError('B is not square')
    
    L = kron(A, np.eye(B.shape[0]))
    R = kron(B, np.eye(A.shape[0]))
    
    return L+R

In [35]:
print(kron(big_test_matrix[:2, :2], big_test_matrix[2:4, 2:4]))

[[ 1  2  2  4]
 [ 3  4  6  8]
 [ 3  6  4  8]
 [ 9 12 12 16]]


In [36]:
n = 3

In [37]:
# Входной BMAP
matrD_0 = np.matrix([[-0.2, 0.6], [0.1, -0.4]])
matrD = np.matrix([[85, 0.99], [0.2, 2.54]])
matrD_k = [matrD_0]

W_ = matrD_0.shape[0]
W = W_ - 1

q = 0.8
for k in range(1, n+1):
    matrD_k.append(matrD * (q ** (k-1)) * (1 - q) / (1 - q ** 3))

print(matrD_0)

[[-0.2  0.6]
 [ 0.1 -0.4]]


In [38]:
# Поток поломок MMAP
matrH0 = np.matrix([[-8.110725, 0], [0, -0.26325]])
matrH1 = np.matrix([[8.0568, 0.053925], [0.146625, 0.116625]])
Z_ = matrH1.shape[0]
Z = Z_ - 1

In [39]:
# Поток обслуживания PH1
beta1 = np.matrix([1, 0])
matrS1 = np.matrix([[-20, 20], [0, -20]])
M1 = matrS1.shape[0]
M1_ = M1 + 1
M1_e = np.matrix([[1], [1]])
matrS1_0 = - matrS1 * M1_e

In [40]:
# Поток обслуживания PH2
beta2 = np.matrix([1, 0])
matrS2 = np.matrix([[-15, 15], [0, -15]])
M2 = matrS2.shape[0]
M2_ = M2 + 1
M2_e = np.matrix([[1], [1]])
matrS2_0 = - matrS2 * M2_e

In [41]:
matrS_w = kron(matrS1_0 * beta1, M2_e * beta2) + kron(M1_e * beta1, matrS2_0 * beta2)

In [42]:
# Поток ремонта PH
tau = np.matrix([0.05, 0.95])
matrT = np.matrix([[-1.86075, 0], [0, -146.9994]])
T_e = np.matrix([[1],[1]])
matrT0 = - matrT * (T_e)

R = matrT.shape[0]
R_ = R + 1

In [43]:
# Q_00
matrQ_00 = np.bmat([[kronsum(kronsum(matrT, matrH0 + matrH1), matrD_k[0]), kron(kron(matrT0, np.eye(Z_)), np.eye(W_))],
                    [kron(kron(tau, matrH1), np.eye(W_)), kronsum(matrH0, matrD_k[0])]])
print(matrQ_00)

[[ -2.11467500e+00  -0.00000000e+00  -0.00000000e+00  -0.00000000e+00
    6.53925000e-01   0.00000000e+00   0.00000000e+00   0.00000000e+00
    1.86075000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [ -0.00000000e+00  -2.11467500e+00  -0.00000000e+00  -0.00000000e+00
    0.00000000e+00   6.53925000e-01   0.00000000e+00   0.00000000e+00
    0.00000000e+00   1.86075000e+00   0.00000000e+00   0.00000000e+00]
 [ -0.00000000e+00  -0.00000000e+00  -2.11467500e+00  -0.00000000e+00
    0.00000000e+00   0.00000000e+00   6.53925000e-01   0.00000000e+00
    0.00000000e+00   0.00000000e+00   1.86075000e+00   0.00000000e+00]
 [ -0.00000000e+00  -0.00000000e+00  -0.00000000e+00  -2.11467500e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   6.53925000e-01
    0.00000000e+00   0.00000000e+00   0.00000000e+00   1.86075000e+00]
 [  2.46625000e-01   0.00000000e+00   0.00000000e+00   0.00000000e+00
   -1.47546025e+02  -0.00000000e+00  -0.00000000e+00  -0.00000000e+00
    1.46999400e+

In [44]:
# Q_10
matrQ_10 = la.block_diag(kron(kron(kron(np.eye(R), matrS2_0), np.eye(Z_)), np.eye(W_)), 
                         kron(kron((kron(matrS1_0, M1_e) + kron(M2_e, matrS2_0)),
                                   np.eye(Z_)), np.eye(W_)))

In [45]:
# Q_0
matrQ_0 = la.block_diag(kron(kron(kron(np.eye(R), matrS2_0 * beta2), np.eye(Z_)), np.eye(W_)),
                        kron(kron(matrS_w, np.eye(Z_)), np.eye(W_)))
pd.DataFrame(matrQ_0)

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
print(kronsum(kronsum(kronsum(matrS1, matrS2), matrH0), matrD_0).shape)

(16, 16)


In [51]:
# Q_1
matrQ_1 = np.bmat([[kronsum(kronsum(kronsum(matrT, matrS2), (matrH0 + matrH1)), matrD_k[0]),
                     kron(kron(kron(matrT0 * beta1, np.eye(M2)), np.eye(Z_)), np.eye(W_))],
                    [kron(kron(kron(M1_e * tau, np.eye(M2)), matrH1), np.eye(W_)),
                     kronsum(kronsum(kronsum(matrS1, matrS2), matrH0), matrD_0)]])
print(matrQ_1)
print(beta1)
print(matrT0 * beta1)
# print(kronsum(kronsum(kronsum(matrT, matrS2), (matrH0 + matrH1)), matrD_0))

[[-17.114675  -0.        -0.       ...,   0.         0.         0.      ]
 [ -0.       -17.114675  -0.       ...,   0.         0.         0.      ]
 [ -0.        -0.       -17.114675 ...,   0.         0.         0.      ]
 ..., 
 [  0.         0.         0.       ..., -35.66325   -0.        -0.      ]
 [  0.         0.         0.       ...,  -0.       -35.66325   -0.      ]
 [  0.         0.         0.       ...,  -0.        -0.       -35.66325 ]]
[[1 0]]
[[   1.86075    0.     ]
 [ 146.9994     0.     ]]


In [48]:
matrQ_0k = [matrQ_00]
for k in range(1, n+1):
    matr_temp = la.block_diag(kron(kron(kron(np.eye(R), beta2), np.eye(Z_)), matrD_k[k]),
                              kron(kron(kron(beta1, beta2), np.eye(Z_)), matrD_k[k]))
    matrQ_0k.append(matr_temp)

In [49]:
matrQ_k = [matrQ_0, matrQ_1]
for k in range(2, n+1):
    matr_temp = la.block_diag(kron(kron(kron(np.eye(R), np.eye(M2)), np.eye(Z_)), matrD_k[k-1]),
                              kron(kron(kron(np.eye(M1), np.eye(M2)), np.eye(Z_)), matrD_k[k-1]))
    matrQ_k.append(matr_temp)

In [49]:
np.array()

TypeError: Required argument 'object' (pos 1) not found